## Trataremos los datos con la tecnica Boosting:

    - Trabajaremos con la misma data sobre el diabetes.
    - las verificaciones ya fueron realizadas previamente, no tiene valores duplicados ni faltantes.

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import *
from imblearn.metrics import specificity_score
from pickle import dump

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv")

In [3]:
X = data.drop(['Outcome'], axis=1)
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [21]:
modelo = XGBClassifier()
modelo.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [22]:
test_pred = modelo.predict(X_test)
train_pred = modelo.predict(X_train)

In [23]:
get_metrics(y_train, y_test, train_pred, test_pred)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
Test,0.707792,0.615385,0.69596,0.580645,0.654545,0.737374
Diferencia,0.292208,0.384615,0.30404,0.419355,0.345455,0.262626


### Observaciones: 

- Podemos observar que el modelo de entrenamiento, tiene un desempeño del 100%, lo cual nos sugiere que deberiamos verificar los hiperparametros para mejorar el modelo.
- La diferencia porcentual es bastante alta entre el grupo de testeo y entrenamiento.

In [26]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Suponiendo que ya tienes tus conjuntos de entrenamiento (X_train, y_train)

# Definir el modelo XGBClassifier
xgb_model = XGBClassifier()

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

# Configurar la búsqueda en cuadrícula
grid_search = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    scoring='accuracy',  # Puedes usar la métrica que desees optimizar
    cv=5,  # Número de divisiones en la validación cruzada
    verbose=1,
    n_jobs=-1  # Utiliza todos los núcleos disponibles
)

# Realizar la búsqueda en cuadrícula en tus datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Obtener el modelo con los mejores hiperparámetros
mejor_modelo_xgb = grid_search.best_estimator_


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Mejores hiperparámetros: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}


In [27]:
# Supongamos que ya tienes tus conjuntos de prueba (X_test, y_test)

# Realizar predicciones en el conjunto de prueba utilizando el mejor modelo
y_pred = mejor_modelo_xgb.predict(X_test)

# Calcular la precisión del modelo en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy en conjunto de prueba: {accuracy}')



Accuracy en conjunto de prueba: 0.7662337662337663


In [28]:
test_pred_op = mejor_modelo_xgb.predict(X_test)
train_pred_op = mejor_modelo_xgb.predict(X_train)

In [29]:
get_metrics(y_train, y_test, train_pred_op, test_pred_op)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.846906,0.763819,0.815660,0.821622,0.713615,0.917706
Test,0.766234,0.666667,0.741414,0.679245,0.654545,0.828283
Diferencia,0.080672,0.097152,0.074246,0.142376,0.059070,0.089423


In [32]:
dump(mejor_modelo_xgb, open("/workspaces/Carlos2607-a-Boosting/models/mejor_modelo.sav", "wb"))

### Teniendo en cuenta los 3 modelos que hemos usado para predecir los datos, podemos determinar que el metodo de random forest presenta una mejora significativa, ya que tiene los margenes de diferencia mas reducidos entre el grupo de entranamiento y prueba.

